In [1]:
import pandas as pd
import requests

In [257]:
# input for file naming convention
initial_value = "11.17.20_v3_us.10.mo.toroid_covidcast&policy&ML"

In [258]:
from datetime import date

In [259]:
# pull updated data from covidcast API csv file
full_symp = pd.read_csv("data_state.covidcast_11.16.20.csv")

In [260]:
full_symp = full_symp.drop('Unnamed: 0', axis=1)

In [261]:
full_symp['time_value'] = pd.to_datetime(full_symp['time_value'])

In [262]:
full_symp['daym'] = full_symp['time_value'].dt.day
full_symp['monthn'] = full_symp['time_value'].dt.month

In [263]:
full_symp = full_symp.drop('state_lower', axis=1)

In [264]:
df1 = full_symp.loc[full_symp['monthn'] == 11]

In [265]:

#
df1 = df1[['geo_value', 'jhu_d', 'jhu_case', 'monthn']].groupby(['geo_value', 'monthn']).sum()

In [266]:
df1 = df1.reset_index(drop=True)

In [267]:
df1 = df1.sort_values(by='jhu_case')

In [268]:
#df1.head(20)

In [269]:
df1.shape

(52, 2)

In [270]:
sname = pd.read_csv('state_ll.csv')

In [271]:
dflatlon = sname.merge(full_symp[['geo_value', 'latitude', 'longitude', 'monthn']], how='inner', left_on=['state_lower'], right_on = ['geo_value'])

In [272]:
dflatlon = dflatlon.drop(['state_lower', 'state', 'latitude_y', 'longitude_y'], axis=1)

In [273]:
dfl2 = dflatlon.drop_duplicates(subset=['geo_value', 'monthn']).reset_index(drop=True)

In [274]:
dfl2.rename(columns={'latitude_x':'latitude'}, inplace=True)

In [275]:
dfl2.rename(columns={'longitude_x':'longitude'}, inplace=True)

In [276]:
dfl2['z'] = (dfl2['monthn']*9) - 12

In [277]:
dpmlist = [28, 31, 30, 31, 30, 31, 31, 30, 31, 30]
monlist = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
dfc = pd.DataFrame()
dfc['dpm'] = dpmlist
dfc['mon'] = monlist

In [278]:
dfl2 = dfl2.merge(dfc, left_on=['monthn'], right_on=['mon'])
dfl2 = dfl2.drop('mon', axis=1)

In [279]:
dfl2 = dfl2.loc[(dfl2['name'] != 'Alaska')]
dfl2 = dfl2.loc[(dfl2['name'] != 'Hawaii')]
dfl2 = dfl2.loc[(dfl2['name'] != 'Puerto Rico')]

In [280]:
dfl2 = dfl2.loc[(dfl2['name'] == 'Arizona')| 
                (dfl2['name'] == 'Florida')|
                (dfl2['name'] == 'New York')|
                (dfl2['name'] == 'Arkansas')|
                (dfl2['name'] == 'South Dakota')|
                (dfl2['name'] == 'Georgia')|
                (dfl2['name'] == 'North Dakota')|
                (dfl2['name'] == 'Washington')|
                (dfl2['name'] == 'North Carolina')|
                (dfl2['name'] == 'West Virginia')
               ]

In [281]:
def make_interpolater(left_min, left_max, right_min, right_max): 
    # Figure out how 'wide' each range is  
    leftSpan = left_max - left_min  
    rightSpan = right_max - right_min  

    # Compute the scale factor between left and right values 
    scaleFactor = float(rightSpan) / float(leftSpan) 

    # create interpolation function using pre-calculated scaleFactor
    def interp_fn(value):
        return right_min + (value-left_min)*scaleFactor

    return interp_fn

In [282]:
for column in full_symp[["survey_c", "survey_i", 'c_tests', 'hv', 'doc_visit', "move_home", "move_partw", "move_fullw", "search"]]:
    col_min = full_symp[column].min() 
    col_max = full_symp[column].max() 
    col_list = full_symp[column].tolist()
    scalar = make_interpolater(col_min + .0002, col_max, .7, 4.0) 
    col_interp = [scalar(x) for x in col_list] 
    full_symp[(str(column) + str('_scale'))] = col_interp

In [283]:
col_min = dfl2['latitude'].min() 
col_max = dfl2['latitude'].max() 
col_list = dfl2['latitude'].tolist()
scalar = make_interpolater(24.646359354721024, 50.75803143090923, -90, 90) 
col_interp = [scalar(x) for x in col_list] 
dfl2["lat_scale"] = col_interp

In [284]:
col_min = dfl2['longitude'].min() 
col_max = dfl2['longitude'].max() 
col_list = dfl2['longitude'].tolist()
scalar = make_interpolater(-128.26540093390213, -61.92020706609784, -180, 180) 
col_interp = [scalar(x) for x in col_list] 
dfl2["lon_scale"] = col_interp

In [285]:
col_min = full_symp['jhu_d'].min() 
col_max = full_symp['jhu_d'].max() 
col_list = full_symp['jhu_d'].tolist()
scalar = make_interpolater(col_min + .0002, col_max, .5, 30) 
col_interp = [scalar(x) for x in col_list] 
full_symp["jhu_d_scale"] = col_interp

In [286]:
col_min = full_symp['jhu_case'].min() 
col_max = full_symp['jhu_case'].max() 
col_list = full_symp['jhu_case'].tolist()
scalar = make_interpolater(col_min + .0002, col_max, .5, 20) 
col_interp = [scalar(x) for x in col_list] 
full_symp["jhu_case_scale"] = col_interp

## Policy dataframe

In [287]:
url = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest_withnotes.csv'
dfpolicy = pd.read_csv(url, dtype={'RegionName':'object', 'RegionCode':'object'})

In [288]:
dfpolicy['Date'] = dfpolicy['Date'].astype(str)
dfpolicy["date"] = pd.to_datetime(dfpolicy['Date'], format='%Y%m%d')

In [289]:
dfpolicy.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode', 'Date',
       'C1_School closing', 'C1_Flag', 'C1_Notes', 'C2_Workplace closing',
       'C2_Flag', 'C2_Notes', 'C3_Cancel public events', 'C3_Flag', 'C3_Notes',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C4_Notes',
       'C5_Close public transport', 'C5_Flag', 'C5_Notes',
       'C6_Stay at home requirements', 'C6_Flag', 'C6_Notes',
       'C7_Restrictions on internal movement', 'C7_Flag', 'C7_Notes',
       'C8_International travel controls', 'C8_Notes', 'E1_Income support',
       'E1_Flag', 'E1_Notes', 'E2_Debt/contract relief', 'E2_Notes',
       'E3_Fiscal measures', 'E3_Notes', 'E4_International support',
       'E4_Notes', 'H1_Public information campaigns', 'H1_Flag', 'H1_Notes',
       'H2_Testing policy', 'H2_Notes', 'H3_Contact tracing', 'H3_Notes',
       'H4_Emergency investment in healthcare', 'H4_Notes',
       'H5_Investment in vaccines', 'H5_Notes', 'H6_Facial Coverings',
       'H6_Flag', 'H6_

In [290]:
dfpolicy.iloc[15000:15002]

,CountryName,CountryCode,RegionName,RegionCode,Date,C1_School closing,C1_Flag,C1_Notes,C2_Workplace closing,C2_Flag,...,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay,date
15000,Germany,DEU,NaN,NaN,20200707,1.0,1.0,This is decided at the level of the federal st...,2.0,0.0,...,55.09,63.1,63.1,58.63,58.63,62.15,62.15,37.5,37.5,2020-07-07
15001,Germany,DEU,NaN,NaN,20200708,1.0,1.0,NaN,2.0,0.0,...,55.09,63.1,63.1,58.63,58.63,62.15,62.15,37.5,37.5,2020-07-08


In [291]:
m = str("nonote")
dfpolicy = dfpolicy.fillna(5)

In [292]:
dfp.loc[(dfp['RegionName'] == 'Arkansas') & (dfp[['C1_Notes', 'C2_Notes', 'C3_Notes', 'C4_Notes', 'C5_Notes', 'C6_Notes', 'C7_Notes', 'C8_Notes']] != 'nonote').any(1)]

,CountryName,CountryCode,RegionName,RegionCode,Date,C1_School closing,C1_Flag,C1_Notes,C2_Workplace closing,C2_Flag,...,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay,date
59602,United States,USA,Arkansas,US_AR,20200202,0,nonote,nonote,0,nonote,...,5.56,7.14,7.14,9.52,9.52,11.11,11.11,0,0,2020-02-02
59631,United States,USA,Arkansas,US_AR,20200302,0,nonote,nonote,0,nonote,...,8.33,10.71,10.71,11.31,11.31,13.19,13.19,0,0,2020-03-02
59641,United States,USA,Arkansas,US_AR,20200312,1,0,"On 12 March 2020, the governor recommended the...",1,1,...,19.44,30.95,30.95,18.45,18.45,21.53,21.53,0,0,2020-03-12
59642,United States,USA,Arkansas,US_AR,20200313,1,0,"On 12 March 2020, the governor recommended the...",1,1,...,26.39,36.67,36.67,22.92,22.92,26.74,26.74,0,0,2020-03-13
59643,United States,USA,Arkansas,US_AR,20200314,1,0,"On 12 March 2020, the governor recommended the...",1,1,...,26.39,36.67,36.67,22.92,22.92,26.74,26.74,0,0,2020-03-14
59644,United States,USA,Arkansas,US_AR,20200315,1,0,"On 12 March 2020, the governor recommended the...",1,1,...,26.39,36.67,36.67,22.92,22.92,26.74,26.74,0,0,2020-03-15
59645,United States,USA,Arkansas,US_AR,20200316,1,1,"On 15 March 2020, the governor allows schools ...",1,1,...,46.3,60.24,60.24,35.71,35.71,41.67,41.67,0,0,2020-03-16
59646,United States,USA,Arkansas,US_AR,20200317,3,1,"On 16 March 2020, the governor order schools t...",2,1,...,57.41,70.95,70.95,48.21,48.21,50,50,37.5,37.5,2020-03-17
59647,United States,USA,Arkansas,US_AR,20200318,3,1,"On 16 March 2020, the governor order schools t...",2,1,...,57.41,70.95,70.95,48.21,48.21,50,50,37.5,37.5,2020-03-18
59648,United States,USA,Arkansas,US_AR,20200319,3,1,"On 16 March 2020, the governor order schools t...",2,1,...,57.41,70.95,70.95,48.21,48.21,50,50,37.5,37.5,2020-03-19


In [293]:
full_symp = full_symp.merge(sname[['name', 'state_lower']], how='inner', left_on=['geo_value'], right_on=['state_lower'])

In [294]:
full_symp = full_symp.drop('state_lower', axis=1)

In [295]:
full_symp = full_symp.merge(dfpolicy, how='left', left_on=['name_y', 'time_value'], right_on=['RegionName', 'date'])

In [296]:
full_symp = full_symp.drop(['CountryName', 'RegionName', 'date', 'name_y'], axis=1)
full_symp.rename(columns={'name_x':'name'}, inplace=True)

In [297]:
full_symp = full_symp.fillna(0)

In [298]:
full_symp.isnull().values.any()

False

In [299]:
full_symp['C1_chg'] = full_symp['C1_School closing'].diff()

In [300]:
full_symp['C2_chg'] = full_symp['C2_Workplace closing'].diff()
full_symp['C3_chg'] = full_symp['C3_Cancel public events'].diff()
full_symp['C4_chg'] = full_symp['C4_Restrictions on gatherings'].diff()
full_symp['C5_chg'] = full_symp['C5_Close public transport'].diff()
full_symp['C6_chg'] = full_symp['C6_Stay at home requirements'].diff()
full_symp['C7_chg'] = full_symp['C7_Restrictions on internal movement'].diff()
full_symp['C8_chg'] = full_symp['C8_International travel controls'].diff()

In [301]:
full_symp['E1_chg'] = full_symp['E1_Income support'].diff()
full_symp['E2_chg'] = full_symp['E2_Debt/contract relief'].diff()
full_symp['E3_chg'] = full_symp['E3_Fiscal measures'].diff()

In [302]:
full_symp['H1_chg'] = full_symp['H1_Public information campaigns'].diff()
full_symp['H2_chg'] = full_symp['H2_Testing policy'].diff()
full_symp['H3_chg'] = full_symp['H3_Contact tracing'].diff()
full_symp['H4_chg'] = full_symp['H4_Emergency investment in healthcare'].diff()
full_symp['H5_chg'] = full_symp['H5_Investment in vaccines'].diff()
full_symp['H6_chg'] = full_symp['H6_Facial Coverings'].diff()

In [303]:
fs = (full_symp.loc[(full_symp['name'] == 'North Dakota') & 
                                              (full_symp['monthn'] == 9)])

In [304]:
(full_symp[['C1_Notes', 'C2_Notes', 'C3_Notes', 'C4_Notes', 
                                                          'C5_Notes', 'C6_Notes', 'C7_Notes', 'C8_Notes']] 
                                               != 'nonote').any(1) & 

SyntaxError: invalid syntax (<ipython-input-304-e348280c868a>, line 3)

In [305]:
fs.to_csv('north.dakota.check.csv')

In [306]:
print(list(full_symp.columns))

['geo_value', 'time_value', 'search', 'doc_visit', 'survey_c', 'survey_i', 'c_tests', 'hv', 'jhu_case', 'jhu_d', 'move_home', 'move_fullw', 'move_partw', 'latitude', 'longitude', 'name', 'daym', 'monthn', 'survey_c_scale', 'survey_i_scale', 'c_tests_scale', 'hv_scale', 'doc_visit_scale', 'move_home_scale', 'move_partw_scale', 'move_fullw_scale', 'search_scale', 'jhu_d_scale', 'jhu_case_scale', 'CountryCode', 'RegionCode', 'Date', 'C1_School closing', 'C1_Flag', 'C1_Notes', 'C2_Workplace closing', 'C2_Flag', 'C2_Notes', 'C3_Cancel public events', 'C3_Flag', 'C3_Notes', 'C4_Restrictions on gatherings', 'C4_Flag', 'C4_Notes', 'C5_Close public transport', 'C5_Flag', 'C5_Notes', 'C6_Stay at home requirements', 'C6_Flag', 'C6_Notes', 'C7_Restrictions on internal movement', 'C7_Flag', 'C7_Notes', 'C8_International travel controls', 'C8_Notes', 'E1_Income support', 'E1_Flag', 'E1_Notes', 'E2_Debt/contract relief', 'E2_Notes', 'E3_Fiscal measures', 'E3_Notes', 'E4_International support', 'E4_No

In [307]:
full_symp.head()

,geo_value,time_value,search,doc_visit,survey_c,survey_i,c_tests,hv,jhu_case,jhu_d,...,C8_chg,E1_chg,E2_chg,E3_chg,H1_chg,H2_chg,H3_chg,H4_chg,H5_chg,H6_chg
0,ak,2020-02-01,0.000000,0.133990,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ak,2020-02-02,0.000000,0.189365,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ak,2020-02-03,0.000000,0.227790,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ak,2020-02-04,0.000000,0.187205,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ak,2020-02-05,25.435233,0.071758,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [308]:
statelist = full_symp['geo_value'].unique().tolist()
monthn = []
geo_value = []
weekn = []
s_mean = []
g_mean = []
c_mean = []
e_mean = []
for s in statelist:
    
    for i in monlist:
        dfw = full_symp.loc[(full_symp['geo_value'] == s) & (full_symp['monthn'] == i) & (full_symp['daym'] < 8)]
        val_wk1_inA = dfw['StringencyIndex'].mean()
        val_wk1_inB = dfw['GovernmentResponseIndex'].mean()
        val_wk1_inC = dfw['ContainmentHealthIndex'].mean()
        val_wk1_inD = dfw['EconomicSupportIndex'].mean()
        monthn.append(i)
        geo_value.append(s)
        weekn.append(1)
        s_mean.append(val_wk1_inA)
        g_mean.append(val_wk1_inB)
        c_mean.append(val_wk1_inC)
        e_mean.append(val_wk1_inD)
        
        dfw2 = full_symp.loc[(full_symp['geo_value'] == s) & (full_symp['monthn'] == i) & (full_symp['daym'] >= 8) & (full_symp['daym'] < 15)]
        val_wk2_inA = dfw['StringencyIndex'].mean()
        val_wk2_inB = dfw['GovernmentResponseIndex'].mean()
        val_wk2_inC = dfw['ContainmentHealthIndex'].mean()
        val_wk2_inD = dfw['EconomicSupportIndex'].mean()
        monthn.append(i)
        geo_value.append(s)
        weekn.append(2)
        s_mean.append(val_wk2_inA)
        g_mean.append(val_wk2_inB)
        c_mean.append(val_wk2_inC)
        e_mean.append(val_wk2_inD)
        
        dfw3 = full_symp.loc[(full_symp['geo_value'] == s) & (full_symp['monthn'] == i) & (full_symp['daym'] >= 15) & (full_symp['daym'] < 22)]
        val_wk3_inA = dfw['StringencyIndex'].mean()
        val_wk3_inB = dfw['GovernmentResponseIndex'].mean()
        val_wk3_inC = dfw['ContainmentHealthIndex'].mean()
        val_wk3_inD = dfw['EconomicSupportIndex'].mean()
        monthn.append(i)
        geo_value.append(s)
        weekn.append(3)
        s_mean.append(val_wk3_inA)
        g_mean.append(val_wk3_inB)
        c_mean.append(val_wk3_inC)
        e_mean.append(val_wk3_inD)
        
        dfw4 = full_symp.loc[(full_symp['geo_value'] == s) & (full_symp['monthn'] == i) & (full_symp['daym'] >=22)]
        val_wk4_inA = dfw['StringencyIndex'].mean()
        val_wk4_inB = dfw['GovernmentResponseIndex'].mean()
        val_wk4_inC = dfw['ContainmentHealthIndex'].mean()
        val_wk4_inD = dfw['EconomicSupportIndex'].mean()
        monthn.append(i)
        geo_value.append(s)
        weekn.append(4)
        s_mean.append(val_wk4_inA)
        g_mean.append(val_wk4_inB)
        c_mean.append(val_wk4_inC)
        e_mean.append(val_wk4_inD)
        

In [309]:
dfindex = pd.DataFrame(
    {'geo_value': geo_value,
    'monthn': monthn,
    'weekn': weekn,
    's_mean': s_mean,
    'g_mean': g_mean,
    'c_mean': c_mean,
    'e_mean': e_mean
    })

In [310]:
dfindex.head()

,geo_value,monthn,weekn,s_mean,g_mean,c_mean,e_mean
0,ak,2,1,11.111429,13.098571,15.278571,0.0
1,ak,2,2,11.111429,13.098571,15.278571,0.0
2,ak,2,3,11.111429,13.098571,15.278571,0.0
3,ak,2,4,11.111429,13.098571,15.278571,0.0
4,ak,3,1,19.044286,18.195714,21.231429,0.0


In [311]:
for column in dfindex[['s_mean', 'g_mean', 'c_mean', 'e_mean']]:
    col_min = dfindex[column].min() 
    col_max = dfindex[column].max() 
    col_list = dfindex[column].tolist()
    scalar = make_interpolater(col_min + .0002, col_max, .2, 2.5) 
    col_interp = [scalar(x) for x in col_list] 
    dfindex[(str(column) + str("_scale"))] = col_interp

In [312]:
dfer = pd.read_csv('xgb_knn_svm_cases_20_days.csv')

In [313]:
dfer = dfer.drop('Unnamed: 0', axis=1)

In [314]:
collist = ['date', 'state', 'xgb', 'knn', 'svm', 'ac']
dfer.columns = collist

In [315]:
dfer['xgb_e'] = (abs(dfer['xgb'] - dfer['ac']) / dfer['ac']) 

In [316]:
dfer['knn_e'] = (abs(dfer['knn'] - dfer['ac']) / dfer['ac']) 

In [317]:
dfer['svm_e'] = (abs(dfer['svm'] - dfer['ac']) / dfer['ac']) 

In [318]:
dfer['date'] = pd.to_datetime(dfer['date'])

In [319]:
for column in dfer[['xgb_e', 'knn_e', 'svm_e']]:
    col_min = dfer[column].min() 
    col_max = dfer[column].max() 
    col_list = dfer[column].tolist()
    scalar = make_interpolater(col_min + .0002, col_max, 1, 3.5) 
    col_interp = [scalar(x) for x in col_list] 
    dfer[(str(column) + str("_scale"))] = col_interp

In [320]:
full_symp = full_symp.merge(dfer, how='left', left_on=['geo_value', 'time_value'], right_on=['state', 'date'])

In [321]:
full_symp = full_symp.fillna(0)

In [322]:
# set DEBUG flag
DEBUG = 0

# Initialize all ANTz Node variables
# You shouldn't need to change anything in this section
# you can find an explanation of these terms in Appendix B of the user manual
id = 0
type = 5
data = id
selected = 0
parent_id = 0
branch_level = 0
child_id = id
child_index = 0
palette_id = 0
ch_input_id = 0
ch_output_id = 0
ch_last_updated = 0
average = 0
samples = 0
aux_a_x = 30
aux_a_y = 30
aux_a_z = 0
aux_b_x = 0
aux_b_y = 0
aux_b_z = 0
color_shift = 0
rotate_vec_x = 0
rotate_vec_y = 0
rotate_vec_z = 0
rotate_vec_s = 1
scale_x = 0
scale_y = 0
scale_z = 0
translate_x = 0
translate_y = 0
translate_z = 0
tag_offset_x = 0
tag_offset_y = 0
tag_offset_z = 0
rotate_rate_x = 0
rotate_rate_y = 0
rotate_rate_z = 0
rotate_x = 0
rotate_y = 0
rotate_z = 0
scale_rate_x = 0
scale_rate_y = 0
scale_rate_z = 0
translate_rate_x = 0
translate_rate_y = 0
translate_rate_z = 0
translate_vec_x = 0
translate_vec_y = 0
translate_vec_z = 0
shader = 0
geometry = 7
line_width = 1
point_size = 0
ratio = 0.1
color_index = 0
color_r = 110
color_g = 110
color_b = 110
color_a = 255
color_fade = 0
texture_id = 0
hide = 0
freeze = 0
topo = 3
facet = 0
auto_zoom_x = 0
auto_zoom_y = 0
auto_zoom_z = 0
trigger_hi_x = 0
trigger_hi_y = 0
trigger_hi_z = 0
trigger_lo_x = 0
trigger_lo_y = 0
trigger_lo_z = 1
set_hi_x = 0
set_hi_y = 0
set_hi_z = 0
set_lo_x = 0
set_lo_y = 0
set_lo_z = 0
proximity_x = 0
proximity_y = 0
proximity_z = 0
proximity_mode_x = 0
proximity_mode_y = 0
proximity_mode_z = 0
segments_x = 20
segments_y = 12
segments_z = 0
tag_mode = 0
format_id = 0
table_id = 0
record_id = id
size = 420

In [323]:
# hr node files
mn = str(initial_value) + "_node.csv"

# hr tag files
mt = str(initial_value) + "_tag.csv"

In [324]:
# Open ANTz Node file for writing
fout = open(mn,"w")

# Open the Tag file to add metadata to ANTz objects
ftag = open(mt,"w")
# Write the header string
outputstring = "id,record_id,table_id,title,description\n"
ftag.write(outputstring)

# Write rows for header, world, camera views (4), and grid to Node file
outputstring = "id,type,data,selected,parent_id,branch_level,child_id,child_index,palette_id,ch_input_id,ch_output_id,ch_last_updated,average,samples,aux_a_x,aux_a_y,aux_a_z,aux_b_x,aux_b_y,aux_b_z,color_shift,rotate_vec_x,rotate_vec_y,rotate_vec_z,rotate_vec_s,scale_x,scale_y,scale_z,translate_x,translate_y,translate_z,tag_offset_x,tag_offset_y,tag_offset_z,rotate_rate_x,rotate_rate_y,rotate_rate_z,rotate_x,rotate_y,rotate_z,scale_rate_x,scale_rate_y,scale_rate_z,translate_rate_x,translate_rate_y,translate_rate_z,translate_vec_x,translate_vec_y,translate_vec_z,shader,geometry,line_width,point_size,ratio,color_index,color_r,color_g,color_b,color_a,color_fade,texture_id,hide,freeze,topo,facet,auto_zoom_x,auto_zoom_y,auto_zoom_z,trigger_hi_x,trigger_hi_y,trigger_hi_z,trigger_lo_x,trigger_lo_y,trigger_lo_z,set_hi_x,set_hi_y,set_hi_z,set_lo_x,set_lo_y,set_lo_z,proximity_x,proximity_y,proximity_z,proximity_mode_x,proximity_mode_y,proximity_mode_z,segments_x,segments_y,segments_z,tag_mode,format_id,table_id,record_id,size\n"
fout.write(outputstring)

# Row for world parameters
outputstring = "1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Row for first camera view
outputstring = "2,1,2,0,0,0,2,2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0.008645,0.825266,-0.564678,1,1,1,-32.446629,-180.908295,143.514175,0,0,1,0,0,0,55.620094,0.600200,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,214.306686,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Row for second camera view
outputstring = "3,1,3,0,2,1,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,1,1,1,-0.500000,0,571.750000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Third camera view
outputstring = "4,1,4,0,2,1,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,-0,1,1,1,0,-90,7,0,0,1,0,0,0,90,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Fourth camera view
outputstring = "5,1,5,0,2,1,5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,-0,-0,1,1,1,85,0,7,0,0,1,0,0,0,90,270,0,0,0,0,0,0,0,-0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Default Grid
outputstring = "6,6,6,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,3,0,0,255,150,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,8,0,0,0,0,0,420\n"
fout.write(outputstring)

# Initialize increments
taginc = 0
id = 6


numlevel1toroids = 1
# month toroid
for index, row in dfl2a.iterrows():
    id += 1
    data = id
    type = 5
    parent_id = 0
    selected = 0
    branch_level = 1
    child_id = id
    translate_x = row['lon_scale']
    translate_y = row['lat_scale']
    translate_z = row['z']
    scale_x = 1 
    scale_y = scale_x
    scale_z = scale_x
    rotate_x = 90
    ratio = .12
    line_width= 2
    geometry = 7
    topo = 3
    color_r = 0
    color_g = 152
    color_b = 255
    color_a = 255
    rotate_x = 0
    rotate_y = 0
    rotate_z = 0
    record_id = id 
    texture_id= 0
    hide = 0

    # Creat output string and write to ANTz Node File
    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
    fout.write(outputstring)

    # Output to Tag file
    tagtext = str(row['name']) + " Month:" + str(row['monthn'])
    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
    ftag.write(tagstring)
    taginc += 1
    
    level1objectid = id
    
    #rod at 90 degrees, holding 4 x indices
    numlevel2toroids = 1
    id += 1
    data = id
    type = 5
    parent_id = level1objectid
    selected = 0
    branch_level = 2
    child_id = id
    translate_x = 90
    translate_y = 0
    translate_z = 370
    scale_x = 2.5 
    scale_y = scale_x
    scale_z = scale_x
    rotate_x = 90
    ratio = .12
    line_width= 2
    geometry = 19
    topo = 6
    color_r = 0
    color_g = 152
    color_b = 255
    color_a = 50
    rotate_x = 0
    rotate_y = 0
    rotate_z = 0
    record_id = id 
    texture_id= 0
    hide = 0

    # Creat output string and write to ANTz Node File
    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
    fout.write(outputstring)
    level2objectid = id
    #supporting two rods
    numlevel3objectid = 2
    for t in range(numlevel3objectid):
        if t == 0:
            id += 1
            data = id
            type = 5
            parent_id = level2objectid
            selected = 0
            branch_level = 3
            child_id = id
            translate_x = 0
            translate_y = 0
            translate_z = 0
            scale_x = 1.55 
            scale_y = scale_x
            scale_z = scale_x
            ratio = .05
            line_width= 1
            geometry = 19
            topo = 6
            color_r = 0
            color_g = 152
            color_b = 255
            color_a = 50
            rotate_x = 90
            rotate_y = 0
            rotate_z = 0
            record_id = id 
            texture_id= 0

            # Creat output string and write to ANTz Node File
            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
            fout.write(outputstring)

            level3objectid = id
            numlevel4objectid = 2
            # two rings per rod
            for r in range(numlevel4objectid):
                if r == 0:
                    id += 1
                    data = id
                    type = 5
                    parent_id = level3objectid
                    selected = 0
                    branch_level = 4
                    child_id = id
                    translate_x = 0
                    translate_y = 0
                    translate_z = 0
                    scale_x = 2.25 
                    scale_y = scale_x
                    scale_z = scale_x
                    ratio = .05
                    line_width= 1
                    geometry = 7
                    topo = 3
                    color_r = 0
                    color_g = 152
                    color_b = 255
                    color_a = 50
                    rotate_x = 0
                    rotate_y = 0
                    rotate_z = 0
                    record_id = id 
                    texture_id= 0

                    # Creat output string and write to ANTz Node File
                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                    fout.write(outputstring)

                    level4objectid = id 
                    numlevel5toroids = ['s_mean', 'g_mean', 'c_mean', 'e_mean']
                    for i, r in dfindex.loc[(dfindex['geo_value'] == row['geo_value']) & 
                                            (dfindex['monthn'] == row['monthn']) &
                                            (dfindex['weekn'] == 1)].iterrows():
                        for ind in numlevel5toroids:
                            id += 1
                            data = id
                            type = 5
                            parent_id = level4objectid
                            selected = 0
                            branch_level = 5
                            child_id = id 
                            if ind == 's_mean':
                                translate_x = 45
                                scale_x = r['s_mean_scale'] 
                                scale_y = scale_x
                                scale_z = scale_x
                                color_r = 230
                                color_g = 132
                                color_b = 60
                                tagtext = str("Stringency Index: ") + str(r['s_mean'])
                            elif ind == 'g_mean':
                                translate_x = 135
                                scale_x = r['g_mean_scale'] 
                                scale_y = scale_x
                                scale_z = scale_x
                                color_r = 52
                                color_g = 165
                                color_b = 214
                                tagtext = str("Government Response Index: ") + str(r['g_mean'])
                            elif ind == 'c_mean':
                                translate_x = -45
                                scale_x = r['c_mean_scale'] 
                                scale_y = scale_x
                                scale_z = scale_x
                                color_r = 9
                                color_g = 184
                                color_b = 32
                                tagtext = str("Containment/ Health Index: ") + str(r['c_mean'])
                            elif ind == 'e_mean':
                                translate_x = -135
                                scale_x = r['e_mean_scale'] 
                                scale_y = scale_x
                                scale_z = scale_x
                                color_r = 45
                                color_g = 78
                                color_b = 240
                                tagtext = str("Economic Support Index: ") + str(r['e_mean'])
                            translate_z = 0
                            rotate_x = 0
                            ratio = .1
                            line_width= 1
                            geometry = 7
                            topo = 3
                            color_a = 255
                            rotate_x = 0
                            rotate_y = 0
                            rotate_z = 0
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)

                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1

                elif r == 1:
                    id += 1
                    data = id
                    type = 5
                    parent_id = level3objectid
                    selected = 0
                    branch_level = 4
                    child_id = id
                    translate_x = -120
                    translate_y = 0
                    translate_z = 0
                    scale_x = 2.25 
                    scale_y = scale_x
                    scale_z = scale_x
                    ratio = .05
                    line_width= 1
                    geometry = 7
                    topo = 3
                    color_r = 0
                    color_g = 152
                    color_b = 255
                    color_a = 50
                    rotate_x = 0
                    rotate_y = 0
                    rotate_z = 0
                    record_id = id 
                    texture_id= 0

                    # Creat output string and write to ANTz Node File
                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                    fout.write(outputstring)

                    level4objectid = id 
                    numlevel5toroids = ['s_mean', 'g_mean', 'c_mean', 'e_mean']
                    for i, r in dfindex.loc[(dfindex['geo_value'] == row['geo_value']) & 
                                            (dfindex['monthn'] == row['monthn']) &
                                            (dfindex['weekn'] == 2)].iterrows():
                        for ind in numlevel5toroids:
                            id += 1
                            data = id
                            type = 5
                            parent_id = level4objectid
                            selected = 0
                            branch_level = 5
                            child_id = id 
                            if ind == 's_mean':
                                translate_x = 45
                                scale_x = r['s_mean_scale'] 
                                scale_y = scale_x
                                scale_z = scale_x
                                color_r = 230
                                color_g = 132
                                color_b = 60
                                tagtext = str("Stringency Index: ") + str(r['s_mean'])
                            elif ind == 'g_mean':
                                translate_x = 135
                                scale_x = r['g_mean_scale'] 
                                scale_y = scale_x
                                scale_z = scale_x
                                color_r = 52
                                color_g = 165
                                color_b = 214
                                tagtext = str("Government Response Index: ") + str(r['g_mean'])
                            elif ind == 'c_mean':
                                translate_x = -45
                                scale_x = r['c_mean_scale'] 
                                scale_y = scale_x
                                scale_z = scale_x
                                color_r = 9
                                color_g = 184
                                color_b = 32
                                tagtext = str("Containment/ Health Index: ") + str(r['c_mean'])
                            elif ind == 'e_mean':
                                translate_x = -135
                                scale_x = r['e_mean_scale'] 
                                scale_y = scale_x
                                scale_z = scale_x
                                color_r = 45
                                color_g = 78
                                color_b = 240
                                tagtext = str("Economic Support Index: ") + str(r['e_mean'])
                            translate_z = 0
                            rotate_x = 0
                            ratio = .1
                            line_width= 1
                            geometry = 7
                            topo = 3
                            color_a = 255
                            rotate_x = 0
                            rotate_y = 0
                            rotate_z = 0
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)

                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1

        elif t == 1:
            id += 1
            data = id
            type = 5
            parent_id = level2objectid
            selected = 0
            branch_level = 3
            child_id = id
            translate_x = 0
            translate_y = 0
            translate_z = 0
            scale_x = 1.55 
            scale_y = scale_x
            scale_z = scale_x
            ratio = .05
            line_width= 1
            geometry = 19
            topo = 6
            color_r = 0
            color_g = 152
            color_b = 255
            color_a = 50
            rotate_x = 90
            rotate_y = 180
            rotate_z = 0
            record_id = id 
            texture_id= 0

            # Creat output string and write to ANTz Node File
            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
            fout.write(outputstring)

            level3objectid = id
            numlevel4objectid = 2
            # two rings per rod
            for r in range(numlevel4objectid):
                if r == 0:
                    id += 1
                    data = id
                    type = 5
                    parent_id = level3objectid
                    selected = 0
                    branch_level = 4
                    child_id = id
                    translate_x = -120
                    translate_y = 0
                    translate_z = 0
                    scale_x = 2.25 
                    scale_y = scale_x
                    scale_z = scale_x
                    ratio = .05
                    line_width= 1
                    geometry = 7
                    topo = 3
                    color_r = 0
                    color_g = 152
                    color_b = 255
                    color_a = 50
                    rotate_x = 0
                    rotate_y = 0
                    rotate_z = 0
                    record_id = id 
                    texture_id= 0

                    # Creat output string and write to ANTz Node File
                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                    fout.write(outputstring)

                    level4objectid = id 
                    numlevel5toroids = ['s_mean', 'g_mean', 'c_mean', 'e_mean']
                    for i, r in dfindex.loc[(dfindex['geo_value'] == row['geo_value']) & 
                                            (dfindex['monthn'] == row['monthn']) &
                                            (dfindex['weekn'] == 3)].iterrows():
                        for ind in numlevel5toroids:
                            id += 1
                            data = id
                            type = 5
                            parent_id = level4objectid
                            selected = 0
                            branch_level = 5
                            child_id = id 
                            if ind == 's_mean':
                                translate_x = 45
                                scale_x = r['s_mean_scale'] 
                                scale_y = scale_x
                                scale_z = scale_x
                                color_r = 230
                                color_g = 132
                                color_b = 60
                                tagtext = str("Stringency Index: ") + str(r['s_mean'])
                            elif ind == 'g_mean':
                                translate_x = 135
                                scale_x = r['g_mean_scale'] 
                                scale_y = scale_x
                                scale_z = scale_x
                                color_r = 52
                                color_g = 165
                                color_b = 214
                                tagtext = str("Government Response Index: ") + str(r['g_mean'])
                            elif ind == 'c_mean':
                                translate_x = -45
                                scale_x = r['c_mean_scale'] 
                                scale_y = scale_x
                                scale_z = scale_x
                                color_r = 9
                                color_g = 184
                                color_b = 32
                                tagtext = str("Containment/ Health Index: ") + str(r['c_mean'])
                            elif ind == 'e_mean':
                                translate_x = -135
                                scale_x = r['e_mean_scale'] 
                                scale_y = scale_x
                                scale_z = scale_x
                                color_r = 45
                                color_g = 78
                                color_b = 240
                                tagtext = str("Economic Support Index: ") + str(r['e_mean'])
                            translate_z = 0
                            rotate_x = 0
                            ratio = .1
                            line_width= 1
                            geometry = 7
                            topo = 3
                            color_a = 255
                            rotate_x = 0
                            rotate_y = 0
                            rotate_z = 0
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)

                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1

                elif r == 1:
                    id += 1
                    data = id
                    type = 5
                    parent_id = level3objectid
                    selected = 0
                    branch_level = 4
                    child_id = id
                    translate_x = 0
                    translate_y = 0
                    translate_z = 0
                    scale_x = 2.25 
                    scale_y = scale_x
                    scale_z = scale_x
                    ratio = .05
                    line_width= 1
                    geometry = 7
                    topo = 3
                    color_r = 0
                    color_g = 152
                    color_b = 255
                    color_a = 50
                    rotate_x = 0
                    rotate_y = 0
                    rotate_z = 0
                    record_id = id 
                    texture_id= 0

                    # Creat output string and write to ANTz Node File
                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                    fout.write(outputstring)

                    level4objectid = id 
                    numlevel5toroids = ['s_mean', 'g_mean', 'c_mean', 'e_mean']
                    for i, r in dfindex.loc[(dfindex['geo_value'] == row['geo_value']) & 
                                            (dfindex['monthn'] == row['monthn']) &
                                            (dfindex['weekn'] == 4)].iterrows():
                        for ind in numlevel5toroids:
                            id += 1
                            data = id
                            type = 5
                            parent_id = level4objectid
                            selected = 0
                            branch_level = 5
                            child_id = id 
                            if ind == 's_mean':
                                translate_x = 45
                                scale_x = r['s_mean_scale'] 
                                scale_y = scale_x
                                scale_z = scale_x
                                color_r = 230
                                color_g = 132
                                color_b = 60
                                tagtext = str("Stringency Index: ") + str(r['s_mean'])
                            elif ind == 'g_mean':
                                translate_x = 135
                                scale_x = r['g_mean_scale'] 
                                scale_y = scale_x
                                scale_z = scale_x
                                color_r = 52
                                color_g = 165
                                color_b = 214
                                tagtext = str("Government Response Index: ") + str(r['g_mean'])
                            elif ind == 'c_mean':
                                translate_x = -45
                                scale_x = r['c_mean_scale'] 
                                scale_y = scale_x
                                scale_z = scale_x
                                color_r = 9
                                color_g = 184
                                color_b = 32
                                tagtext = str("Containment/ Health Index: ") + str(r['c_mean'])
                            elif ind == 'e_mean':
                                translate_x = -135
                                scale_x = r['e_mean_scale'] 
                                scale_y = scale_x
                                scale_z = scale_x
                                color_r = 45
                                color_g = 78
                                color_b = 240
                                tagtext = str("Economic Support Index: ") + str(r['e_mean'])
                            translate_z = 0
                            rotate_x = 0
                            ratio = .1
                            line_width= 1
                            geometry = 7
                            topo = 3
                            color_a = 255
                            rotate_x = 0
                            rotate_y = 0
                            rotate_z = 0
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)

                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1

    numlevel2toroids = row['dpm']
    # day toroid
    for i in range(numlevel2toroids):
        id += 1
        data = id
        type = 5
        parent_id = level1objectid
        selected = 0
        branch_level = 2
        child_id = id
        translate_x = (360/numlevel2toroids)*i
        translate_y = 0
        translate_z = 100
        scale_x = .4
        scale_y = scale_x
        scale_z = scale_x
        line_width= 1
        geometry = 7
        topo = 3
        color_r = 101
        color_g = 154
        color_b = 255
        color_a = 90
        hide = 0
        rotate_x = 0
        rotate_y = 90
        rotate_z = 0
        ratio = .08
        record_id = id 
        texture_id= 0

        # Creat output string and write to ANTz Node File
        outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
        fout.write(outputstring)
        # Output to Tag file
        tagtext = "Day: " + str(i+1) 
        tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
        ftag.write(tagstring)
        taginc += 1
        
        level2objectid = id
        numlevel3toroids = 5
        for index2, row2 in full_symp.loc[(full_symp['geo_value'] == row['geo_value'])& (full_symp['monthn'] == row['monthn']) & (full_symp['daym'] == (i+1))].iterrows():
            for j in range(numlevel3toroids):
                # covid cases, deaths, and ML error cones
                if j == 0:
                    id += 1
                    data = id
                    type = 5
                    parent_id = level2objectid
                    selected = 0
                    branch_level = 3
                    child_id = id
                    translate_x = 130
                    translate_y = 0
                    translate_z = 0
                    scale_x = 6
                    scale_y = scale_x
                    scale_z = scale_x
                    line_width= 1
                    geometry = 19
                    topo = 6
                    color_r = 139
                    color_g = 116
                    color_b = 255
                    color_a = 100
                    rotate_x = 0
                    rotate_y = 0
                    rotate_z = 0
                    ratio = .05
                    record_id = id 
                    texture_id= 0

                    # Creat output string and write to ANTz Node File
                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                    fout.write(outputstring)
                    tagtext = ""
                    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                    ftag.write(tagstring)
                    taginc += 1
                    level3objectid = id
                    numlevel4toroids = 4
                    #parameters

                    for k in range(numlevel4toroids):
                        id += 1
                        data = id
                        type = 5
                        parent_id = level3objectid
                        selected = 0
                        branch_level = 4
                        child_id = id
                        if k == 0:
                            scale_x = row2['jhu_d_scale'] 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = 0
                            geometry = 3
                            topo = 2
                            color_r = 255
                            color_g = 0
                            color_b = 0
                            rotate_x = 90
                            tagtext = "Deaths: " + str(row2['jhu_d'])
                        elif k == 1:
                            scale_x = row2['jhu_case_scale'] 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = -75
                            geometry = 1
                            topo = 1
                            color_r = 166
                            color_g = 0
                            color_b = 89
                            rotate_x = 0
                            tagtext = "Cases: " + str(row2['jhu_case'])
                        elif k == 2:
                            scale_x = row2['xgb_e_scale'] 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = -90
                            geometry = 5
                            topo = 2
                            if (row2['xgb_e'] <= .1056):
                                color_r = 25
                                color_g = 62
                                color_b = 230
                            elif (row2['xgb_e'] <= .1872) & (row2['xgb_e'] > .1056):
                                color_r = 25
                                color_g = 155
                                color_b = 230
                            elif (row2['xgb_e'] <= .2505) & (row2['xgb_e'] > .1872):
                                color_r = 40
                                color_g = 255
                                color_b = 216
                            elif (row2['xgb_e'] <= .3142) & (row2['xgb_e'] > .2505):
                                color_r = 28
                                color_g = 230
                                color_b = 78
                            elif (row2['xgb_e'] <= .3742) & (row2['xgb_e'] > .3142):
                                color_r = 29
                                color_g = 255
                                color_b = 35
                            elif (row2['xgb_e'] <= .4374) & (row2['xgb_e'] > .3742):
                                color_r = 76
                                color_g = 255
                                color_b = 33
                            elif (row2['xgb_e'] <= .5367) & (row2['xgb_e'] > .4374):
                                color_r = 176
                                color_g = 255
                                color_b = 27
                            elif row2['xgb_e'] > .5367:
                                color_r = 255
                                color_g = 252
                                color_b = 31
                            rotate_x = 0
                            tagtext = "XGB Error rate: " + str(row2['xgb_e'])
                        elif k == 3:
                            scale_x = row2['knn_e_scale'] 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = -105
                            geometry = 5
                            topo = 2
                            if (row2['knn_e'] <= .1056):
                                color_r = 25
                                color_g = 62
                                color_b = 230
                            elif (row2['knn_e'] <= .1872) & (row2['knn_e'] > .1056):
                                color_r = 25
                                color_g = 155
                                color_b = 230
                            elif (row2['knn_e'] <= .2505) & (row2['knn_e'] > .1872):
                                color_r = 40
                                color_g = 255
                                color_b = 216
                            elif (row2['knn_e'] <= .3142) & (row2['knn_e'] > .2505):
                                color_r = 28
                                color_g = 230
                                color_b = 78
                            elif (row2['knn_e'] <= .3742) & (row2['knn_e'] > .3142):
                                color_r = 29
                                color_g = 255
                                color_b = 35
                            elif (row2['knn_e'] <= .4374) & (row2['knn_e'] > .3742):
                                color_r = 76
                                color_g = 255
                                color_b = 33
                            elif (row2['knn_e'] <= .5367) & (row2['knn_e'] > .4374):
                                color_r = 176
                                color_g = 255
                                color_b = 27
                            elif row2['knn_e'] > .5367:
                                color_r = 255
                                color_g = 252
                                color_b = 31
                            rotate_x = 0
                            tagtext = "KNN Error rate: " + str(row2['knn_e'])
                        elif k == 4:
                            scale_x = row2['svm_e_scale'] 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = -120
                            geometry = 5
                            topo = 2
                            if (row2['svm_e'] <= .1056):
                                color_r = 25
                                color_g = 62
                                color_b = 230
                            elif (row2['svm_e'] <= .1872) & (row2['svm_e'] > .1056):
                                color_r = 25
                                color_g = 155
                                color_b = 230
                            elif (row2['svm_e'] <= .2505) & (row2['svm_e'] > .1872):
                                color_r = 40
                                color_g = 255
                                color_b = 216
                            elif (row2['svm_e'] <= .3142) & (row2['svm_e'] > .2505):
                                color_r = 28
                                color_g = 230
                                color_b = 78
                            elif (row2['svm_e'] <= .3742) & (row2['svm_e'] > .3142):
                                color_r = 29
                                color_g = 255
                                color_b = 35
                            elif (row2['svm_e'] <= .4374) & (row2['svm_e'] > .3742):
                                color_r = 76
                                color_g = 255
                                color_b = 33
                            elif (row2['svm_e'] <= .5367) & (row2['svm_e'] > .4374):
                                color_r = 176
                                color_g = 255
                                color_b = 27
                            elif row2['svm_e'] > .5367:
                                color_r = 255
                                color_g = 252
                                color_b = 31
                            rotate_x = 0
                            tagtext = "SVM Error rate: " + str(row2['svm_e'])
                        color_a = 255
                        translate_y = 0
                        translate_z = 0
                        rotate_y = 0
                        rotate_z = 0
                        ratio = .1
                        line_width= 1
                        record_id = id 
                        texture_id= 0

                        # Creat output string and write to ANTz Node File
                        outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                        fout.write(outputstring)
                        # Output to Tag file
                        tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                        ftag.write(tagstring)
                        taginc += 1


                # health policies, surveys, doctor and hospital visits, and antigen tests    
                elif j == 1:
                    id += 1
                    data = id
                    type = 5
                    parent_id = level2objectid
                    selected = 0
                    branch_level = 3
                    child_id = id
                    translate_x = 90
                    translate_y = 0
                    translate_z = 0
                    scale_x = 1.5
                    scale_y = scale_x
                    scale_z = scale_x
                    line_width= 1
                    geometry = 19
                    topo = 6
                    color_r = 59
                    color_g = 196
                    color_b = 255
                    color_a = 100
                    rotate_x = 0
                    rotate_y = 0
                    rotate_z = 0
                    ratio = .05
                    record_id = id 
                    texture_id= 0
                    hide = 1

                    # Creat output string and write to ANTz Node File
                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                    fout.write(outputstring)
                    tagtext = ""
                    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                    ftag.write(tagstring)
                    taginc += 1
                    level3objectid = id
                    numlevel4toroids = 7
                    # children of rod at 90 degrees (health policies and toroid)
                    for k in range(numlevel4toroids): 
                        if k == 0:
                            id += 1
                            data = id
                            type = 5
                            parent_id = level3objectid
                            selected = 0
                            branch_level = 4
                            child_id = id
                            scale_x = .35 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = -45
                            translate_y = 0
                            translate_z = 0
                            rotate_x = 90
                            rotate_y = 0
                            rotate_z = 0
                            ratio = .1
                            line_width= 1
                            geometry = 19
                            topo = 4
                            if row2['H2_Testing policy'] == 0:
                                color_r = 230
                                color_g = 234
                                color_b = 234
                                color_a = 255
                            elif row2['H2_Testing policy'] == 1:
                                color_r = 108
                                color_g = 127
                                color_b = 230
                                color_a = 255
                            elif row2['H2_Testing policy'] == 2:
                                color_r = 108
                                color_g = 127
                                color_b = 230
                                color_a = 255
                            elif row2['H2_Testing policy'] == 3:
                                color_r = 68
                                color_g = 0
                                color_b = 199
                                color_a = 255
                            elif row2['H2_Testing policy'] == 5:
                                color_r = 59
                                color_g = 196
                                color_b = 255
                                color_a = 0
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)
                            tagtext = "H2 Testing policy: " + str(row2['H2_Notes'])
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1 
                        elif k == 1:
                            id += 1
                            data = id
                            type = 5
                            parent_id = level3objectid
                            selected = 0
                            branch_level = 4
                            child_id = id
                            scale_x = .35 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = -60
                            translate_y = 0
                            translate_z = 0
                            rotate_x = 90
                            rotate_y = 0
                            rotate_z = 0
                            ratio = .1
                            line_width= 1
                            geometry = 19
                            topo = 4
                            if row2['H3_Contact tracing'] == 0:
                                color_r = 230
                                color_g = 234
                                color_b = 234
                                color_a = 255
                            elif row2['H3_Contact tracing'] == 1:
                                color_r = 108
                                color_g = 127
                                color_b = 230
                                color_a = 255
                            elif row2['H3_Contact tracing'] == 2:
                                color_r = 68
                                color_g = 0
                                color_b = 199
                                color_a = 255
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)
                            tagtext = "H3 Contact Tracing: " + str(row2['H3_Notes'])
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1 
                        elif k == 2:
                            id += 1
                            data = id
                            type = 5
                            parent_id = level3objectid
                            selected = 0
                            branch_level = 4
                            child_id = id
                            scale_x = .35 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = -75
                            translate_y = 0
                            translate_z = 0
                            rotate_x = 90
                            rotate_y = 0
                            rotate_z = 0
                            ratio = .1
                            line_width= 1
                            geometry = 19
                            topo = 4
                            if row2['H4_Emergency investment in healthcare'] == 0:
                                color_r = 230
                                color_g = 234
                                color_b = 234
                                color_a = 255
                            elif row2['H4_Emergency investment in healthcare'] == 5:
                                color_r = 59
                                color_g = 196
                                color_b = 255
                                color_a = 0
                            elif (row2['H4_Emergency investment in healthcare'] < 6250000):
                                color_r = 143
                                color_g = 91
                                color_b = 87
                                color_a = 255
                            elif (row2['H4_Emergency investment in healthcare'] >= 6250000) & (row2['H4_Emergency investment in healthcare'] < 20900000):
                                color_r = 97
                                color_g = 60
                                color_b = 57
                                color_a = 255
                            elif (row2['H4_Emergency investment in healthcare'] >= 20900000) & (row2['H4_Emergency investment in healthcare'] <42000000):
                                color_r = 66
                                color_g = 122
                                color_b = 55
                                color_a = 255
                            elif (row2['H4_Emergency investment in healthcare'] >= 42000000) & (row2['H4_Emergency investment in healthcare'] <175000000):
                                color_r = 240
                                color_g = 201
                                color_b = 62
                                color_a = 255
                            elif (row2['H4_Emergency investment in healthcare'] >= 175000000) & (row2['H4_Emergency investment in healthcare'] <= 1000000000):
                                color_r = 235
                                color_g = 165
                                color_b = 23
                                color_a = 255
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)
                            tagtext = "H4 Emergency investment in healthcare: " + str(row2['H4_Notes'])
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1 
                        elif k == 3:
                            id += 1
                            data = id
                            type = 5
                            parent_id = level3objectid
                            selected = 0
                            branch_level = 4
                            child_id = id
                            scale_x = .35 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = -90
                            translate_y = 0
                            translate_z = 0
                            rotate_x = 90
                            rotate_y = 0
                            rotate_z = 0
                            ratio = .1
                            line_width= 1
                            geometry = 19
                            topo = 4
                            if row2['H5_Investment in vaccines'] == 0:
                                color_r = 230
                                color_g = 234
                                color_b = 234
                                color_a = 255
                            elif row2['H5_Investment in vaccines'] == 5:
                                color_r = 59
                                color_g = 196
                                color_b = 255
                                color_a = 0
                            elif row2['H5_Investment in vaccines'] < 1000:
                                color_r = 143
                                color_g = 91
                                color_b = 87
                                color_a = 255
                            elif row2['H5_Investment in vaccines'] >= 1000:
                                color_r = 235
                                color_g = 165
                                color_b = 23
                                color_a = 255
                            
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)
                            tagtext = "H5 Investment in vaccines'" + str(row2['H5_Notes'])
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1 
                        elif k == 4:
                            id += 1
                            data = id
                            type = 5
                            parent_id = level3objectid
                            selected = 0
                            branch_level = 4
                            child_id = id
                            scale_x = .35 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = -105
                            translate_y = 0
                            translate_z = 0
                            rotate_x = 90
                            rotate_y = 0
                            rotate_z = 0
                            ratio = .1
                            line_width= 1
                            geometry = 19
                            topo = 4
                            if row2['H6_Facial Coverings'] == 0:
                                color_r = 230
                                color_g = 234
                                color_b = 234
                                color_a = 255
                            elif row2['H6_Facial Coverings'] == 1:
                                color_r = 174
                                color_g = 200
                                color_b = 238
                                color_a = 255
                            elif row2['H6_Facial Coverings'] == 2:
                                color_r = 108
                                color_g = 127
                                color_b = 230
                                color_a = 255
                            elif row2['H6_Facial Coverings'] == 3:
                                color_r = 65
                                color_g = 48
                                color_b = 214
                                color_a = 255
                            elif row2['H6_Facial Coverings'] == 4:
                                color_r = 68
                                color_g = 0
                                color_b = 199
                                color_a = 255
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)
                            tagtext = "H6 Facial Coverings" + str(row2['H6_Notes'])
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1 
                        # red geometry alerting to change in policy
                        elif k == 5:
                            id += 1
                            data = id
                            type = 5
                            parent_id = level3objectid
                            selected = 0
                            branch_level = 4
                            child_id = id
                            scale_x = 1.75 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = 170
                            translate_y = 0
                            translate_z = 0
                            rotate_x = 0
                            rotate_y = 0
                            rotate_z = 0
                            ratio = .1
                            line_width= 1
                            geometry = 15
                            topo = 2
                            color_r = 194
                            color_g = 0
                            color_b = 61
                            if row2[['H2_chg', 'H3_chg', 'H4_chg', 'H5_chg', 'H6_chg']].any() > 0:
                                color_a = 255
                            else:
                                color_a = 0
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)
                            tagtext = "Policy change today"
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1 
                        elif k == 6:
                            id += 1
                            data = id
                            type = 5
                            parent_id = level3objectid
                            selected = 0
                            branch_level = 4
                            child_id = id
                            scale_x = 1.2 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = 25
                            translate_y = 0
                            translate_z = 0
                            rotate_x = 90
                            rotate_y = 90
                            rotate_z = 0
                            ratio = .02
                            line_width= 1
                            geometry = 7
                            topo = 3
                            color_r = 59
                            color_g = 196
                            color_b = 255
                            color_a = 50
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)
                            tagtext = ""
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1
                            level4objectid = id
                            numlevel5toroids = 5

                            for m in range(numlevel5toroids): 
                                if m == 0:
                                    id += 1
                                    data = id
                                    type = 5
                                    parent_id = level4objectid
                                    selected = 0
                                    branch_level = 5
                                    child_id = id
                                    scale_x = row2['survey_c_scale'] 
                                    scale_y = scale_x 
                                    scale_z = scale_y
                                    translate_x = 60
                                    translate_y = 0
                                    translate_z = 0
                                    rotate_x = 0
                                    rotate_y = 90
                                    rotate_z = 0
                                    ratio = .1
                                    line_width= 1
                                    geometry = 3
                                    topo = 2
                                    color_r = 0
                                    color_g = 255
                                    color_b = 0
                                    color_a = 255
                                    record_id = id 
                                    texture_id= 0

                                    # Creat output string and write to ANTz Node File
                                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                                    fout.write(outputstring)
                                    # Output to Tag file
                                    tagtext = "symptoms indicate covid-19: " + str(row2["survey_c"])
                                    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                                    ftag.write(tagstring)
                                    taginc += 1
                                elif m == 1:
                                    id += 1
                                    data = id
                                    type = 5
                                    parent_id = level4objectid
                                    selected = 0
                                    branch_level = 5
                                    child_id = id
                                    scale_x = row2['survey_i_scale'] 
                                    scale_y = scale_x
                                    scale_z = scale_y
                                    translate_x = 120
                                    translate_y = 0
                                    translate_z = 0
                                    rotate_x = 0
                                    rotate_y = 90
                                    rotate_z = 0
                                    ratio = .1
                                    line_width= 1
                                    geometry = 3
                                    topo = 2
                                    color_r = 0
                                    color_g = 255
                                    color_b = 255                                
                                    color_a = 255
                                    record_id = id 
                                    texture_id= 0

                                    # Creat output string and write to ANTz Node File
                                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                                    fout.write(outputstring)
                                    # Output to Tag file
                                    tagtext = "symptoms related to influenza: " + str(row2["survey_i"])
                                    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                                    ftag.write(tagstring)
                                    taginc += 1
                                elif m == 2:
                                    id += 1
                                    data = id
                                    type = 5
                                    parent_id = level4objectid
                                    selected = 0
                                    branch_level = 5
                                    child_id = id
                                    scale_x = row2['doc_visit_scale'] 
                                    scale_y = scale_x
                                    scale_z = scale_x
                                    translate_x = -60
                                    translate_y = 0
                                    translate_z = 0
                                    rotate_x = 0
                                    rotate_y = 0
                                    rotate_z = 0
                                    ratio = .1
                                    line_width= 1
                                    geometry = 3
                                    topo = 2
                                    color_r = 70
                                    color_g = 130
                                    color_b = 14
                                    color_a = 255
                                    record_id = id 
                                    texture_id= 0

                                    # Creat output string and write to ANTz Node File
                                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                                    fout.write(outputstring)
                                    # Output to Tag file
                                    tagtext = "doctors visits related to covid: " + str(row2["doc_visit"])
                                    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                                    ftag.write(tagstring)
                                    taginc += 1
                                elif m == 3:
                                    id += 1
                                    data = id
                                    type = 5
                                    parent_id = level4objectid
                                    selected = 0
                                    branch_level = 5
                                    child_id = id
                                    scale_x = row2['hv_scale'] 
                                    scale_y = scale_x
                                    scale_z = scale_x
                                    translate_x = -120
                                    geometry = 3
                                    topo = 2
                                    color_r = 255
                                    color_g = 168
                                    color_b = 0
                                    rotate_y = 0
                                    rotate_z = 0
                                    rotate_x = 0
                                    tagtext = "Hospital visits: " + str(row2['hv'])
                                    record_id = id 
                                    texture_id= 0

                                    # Creat output string and write to ANTz Node File
                                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                                    fout.write(outputstring)
                                    # Output to Tag file
                                    tagtext = "Hospital visits: " + str(row2['hv'])
                                    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                                    ftag.write(tagstring)
                                    taginc += 1
                                elif m == 4:
                                    id += 1
                                    data = id
                                    type = 5
                                    parent_id = level4objectid
                                    selected = 0
                                    branch_level = 5
                                    child_id = id
                                    scale_x = row2['c_tests_scale']
                                    scale_y = scale_x
                                    scale_z = scale_x
                                    translate_x = 180
                                    geometry = 3
                                    topo = 2
                                    color_r = 0
                                    color_g = 255
                                    color_b = 0
                                    rotate_y = 0
                                    rotate_z = 0
                                    rotate_x = 0
                                    record_id = id 
                                    texture_id= 0

                                    # Creat output string and write to ANTz Node File
                                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                                    fout.write(outputstring)
                                    # Output to Tag file
                                    tagtext = "Covid tests: " + str(row2['c_tests'])
                                    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                                    ftag.write(tagstring)
                                    taginc += 1
                                            
                                            
                elif j == 2:
                #economic policy
                    id += 1
                    data = id
                    type = 5
                    parent_id = level2objectid
                    selected = 0
                    branch_level = 3
                    child_id = id
                    translate_x = 60
                    translate_y = 0
                    translate_z = 0
                    scale_x = 1.5
                    scale_y = scale_x
                    scale_z = scale_x
                    line_width= 1
                    geometry = 19
                    topo = 6
                    color_r = 59
                    color_g = 196
                    color_b = 255
                    color_a = 100
                    rotate_x = 0
                    rotate_y = 0
                    rotate_z = 0
                    ratio = .05
                    record_id = id 
                    texture_id= 0

                    # Creat output string and write to ANTz Node File
                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                    fout.write(outputstring)
                    tagtext = ""
                    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                    ftag.write(tagstring)
                    taginc += 1
                    level3objectid = id
                    numlevel4toroids = 4
                    # children of rod at 60 degrees (economic policies and change indicator)
                    for k in range(numlevel4toroids):
                        if k == 0: 
                            id += 1
                            data = id
                            type = 5
                            parent_id = level3objectid
                            selected = 0
                            branch_level = 4
                            child_id = id
                            scale_x = .35 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = -30
                            translate_y = 0
                            translate_z = 0
                            rotate_x = 90
                            rotate_y = 0
                            rotate_z = 0
                            ratio = .1
                            line_width= 1
                            geometry = 19
                            topo = 4
                            if row2['E1_Income support'] == 0:
                                color_r = 230
                                color_g = 234
                                color_b = 234
                                color_a = 255
                            elif row2['E1_Income support'] == 1:
                                color_r = 108
                                color_g = 127
                                color_b = 230
                                color_a = 255
                            elif row2['E1_Income support'] == 2:
                                color_r = 68
                                color_g = 0
                                color_b = 199
                                color_a = 255
                            elif row2['E1_Income support'] == 5:
                                color_r = 59
                                color_g = 196
                                color_b = 255
                                color_a = 0
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)
                            tagtext = "E1 Income support: " + str(row2['E1_Notes'])
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1 
                            
                        if k == 1: 
                            id += 1
                            data = id
                            type = 5
                            parent_id = level3objectid
                            selected = 0
                            branch_level = 4
                            child_id = id
                            scale_x = .35 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = -45
                            translate_y = 0
                            translate_z = 0
                            rotate_x = 90
                            rotate_y = 0
                            rotate_z = 0
                            ratio = .1
                            line_width= 1
                            geometry = 19
                            topo = 4
                            if row2['E2_Debt/contract relief'] == 0:
                                color_r = 230
                                color_g = 234
                                color_b = 234
                                color_a = 255
                            elif row2['E2_Debt/contract relief'] == 1:
                                color_r = 174
                                color_g = 200
                                color_b = 238
                                color_a = 255
                            elif row2['E2_Debt/contract relief'] == 2:
                                color_r = 68
                                color_g = 0
                                color_b = 199
                                color_a = 255
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)
                            tagtext = "E2 Debt/contract relief: " + str(row2['E2_Notes'])
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1 
                            
                        if k == 2: 
                            id += 1
                            data = id
                            type = 5
                            parent_id = level3objectid
                            selected = 0
                            branch_level = 4
                            child_id = id
                            scale_x = .35 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = -60
                            translate_y = 0
                            translate_z = 0
                            rotate_x = 90
                            rotate_y = 0
                            rotate_z = 0
                            ratio = .1
                            line_width= 1
                            geometry = 19
                            topo = 4
                            if row2['E3_Fiscal measures'] == 0:
                                color_r = 230
                                color_g = 234
                                color_b = 234
                                color_a = 255
                            elif row2['E3_Fiscal measures'] == 1:
                                color_r = 108
                                color_g = 127
                                color_b = 230
                                color_a = 255
                            elif (row2['E3_Fiscal measures'] < 4220000):
                                color_r = 143
                                color_g = 91
                                color_b = 87
                                color_a = 255
                            elif (row2['E3_Fiscal measures'] >= 4220000) & (row2['E3_Fiscal measures'] < 19600000):
                                color_r = 97
                                color_g = 60
                                color_b = 57
                                color_a = 255
                            elif (row2['E3_Fiscal measures'] >= 19600000) & (row2['E3_Fiscal measures'] <53000000):
                                color_r = 66
                                color_g = 122
                                color_b = 55
                                color_a = 255
                            elif (row2['E3_Fiscal measures'] >= 53000000) & (row2['E3_Fiscal measures'] <200000000):
                                color_r = 240
                                color_g = 201
                                color_b = 62
                                color_a = 255
                            elif (row2['E3_Fiscal measures'] >= 200000000) & (row2['E3_Fiscal measures'] <= 1000000005):
                                color_r = 235
                                color_g = 165
                                color_b = 23
                                color_a = 255
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)
                            tagtext = "E3 Fiscal measures: " + str(row2['E3_Notes'])
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1 
                    
                        # red geometry alerting to change in policy
                        elif k == 3:
                            id += 1
                            data = id
                            type = 5
                            parent_id = level3objectid
                            selected = 0
                            branch_level = 4
                            child_id = id
                            scale_x = 1.75 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = 170
                            translate_y = 0
                            translate_z = 0
                            rotate_x = 0
                            rotate_y = 0
                            rotate_z = 0
                            ratio = .1
                            line_width= 1
                            geometry = 15
                            topo = 2
                            color_r = 194
                            color_g = 0
                            color_b = 61
                            if row2[['E1_chg', 'E2_chg', 'E3_chg']].any() > 0:
                                color_a = 255
                            else:
                                color_a = 0
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)
                            tagtext = "Policy change today"
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1
                            
                # closure policies, mobile phone movement    
                elif j == 3:
                    id += 1
                    data = id
                    type = 5
                    parent_id = level2objectid
                    selected = 0
                    branch_level = 3
                    child_id = id
                    translate_x = 50
                    translate_y = 0
                    translate_z = 0
                    scale_x = 6
                    scale_y = scale_x
                    scale_z = scale_x
                    line_width= 1
                    geometry = 19
                    topo = 6
                    color_r = 59
                    color_g = 196
                    color_b = 255
                    color_a = 100
                    rotate_x = 0
                    rotate_y = 0
                    rotate_z = 0
                    ratio = .05
                    record_id = id 
                    texture_id= 0
                    hide = 0

                    # Creat output string and write to ANTz Node File
                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                    fout.write(outputstring)
                    tagtext = ""
                    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                    ftag.write(tagstring)
                    taginc += 1
                    level3objectid = id
                    numlevel4toroids = 10
                    for k in range(numlevel4toroids): 
                        if k == 0:
                            id += 1
                            data = id
                            type = 5
                            parent_id = level3objectid
                            selected = 0
                            branch_level = 4
                            child_id = id
                            scale_x = .35 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = -30
                            translate_y = 0
                            translate_z = 0
                            rotate_x = 90
                            rotate_y = 0
                            rotate_z = 0
                            ratio = .1
                            line_width= 1
                            geometry = 19
                            topo = 4
                            if row2['C1_School closing'] == 0:
                                color_r = 230
                                color_g = 234
                                color_b = 234
                                color_a = 255
                            elif row2['C1_School closing'] == 1:
                                color_r = 108
                                color_g = 127
                                color_b = 230
                                color_a = 255
                            elif row2['C1_School closing'] == 2:
                                color_r = 108
                                color_g = 127
                                color_b = 230
                                color_a = 255
                            elif row2['C1_School closing'] == 3:
                                color_r = 68
                                color_g = 0
                                color_b = 199
                                color_a = 255
                            elif row2['C1_School closing'] == 5:
                                color_r = 59
                                color_g = 196
                                color_b = 255
                                color_a = 0
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)
                            tagtext = "C1 School closing: " + str(row2['C1_Notes'])
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1 
                        elif k == 1:
                            id += 1
                            data = id
                            type = 5
                            parent_id = level3objectid
                            selected = 0
                            branch_level = 4
                            child_id = id
                            scale_x = .35 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = -45
                            translate_y = 0
                            translate_z = 0
                            rotate_x = 90
                            rotate_y = 0
                            rotate_z = 0
                            ratio = .1
                            line_width= 1
                            geometry = 19
                            topo = 4
                            if row2['C2_Workplace closing'] == 0:
                                color_r = 230
                                color_g = 234
                                color_b = 234
                                color_a = 255
                            elif row2['C2_Workplace closing'] == 1:
                                color_r = 108
                                color_g = 127
                                color_b = 230
                                color_a = 255
                            elif row2['C2_Workplace closing'] == 2:
                                color_r = 68
                                color_g = 0
                                color_b = 199
                                color_a = 255
                            elif row2['C2_Workplace closing'] == 5:
                                color_r = 59
                                color_g = 196
                                color_b = 255
                                color_a = 0
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)
                            tagtext = "C2_Workplace closing: " + str(row2['C2_Notes'])
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1 
                        elif k == 2:
                            id += 1
                            data = id
                            type = 5
                            parent_id = level3objectid
                            selected = 0
                            branch_level = 4
                            child_id = id
                            scale_x = .35 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = -60
                            translate_y = 0
                            translate_z = 0
                            rotate_x = 90
                            rotate_y = 0
                            rotate_z = 0
                            ratio = .1
                            line_width= 1
                            geometry = 19
                            topo = 4
                            if row2['C3_Cancel public events'] == 0:
                                color_r = 230
                                color_g = 234
                                color_b = 234
                                color_a = 255
                            elif row2['C3_Cancel public events'] == 1:
                                color_r = 108
                                color_g = 127
                                color_b = 230
                                color_a = 255
                            elif row2['C3_Cancel public events'] == 2:
                                color_r = 68
                                color_g = 0
                                color_b = 199
                                color_a = 255
                            elif row2['C3_Cancel public events'] == 5:
                                color_r = 59
                                color_g = 196
                                color_b = 255
                                color_a = 0
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)
                            tagtext = "C3_Cancel public events: " + str(row2['C3_Notes'])
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1 
                        elif k == 3:
                            id += 1
                            data = id
                            type = 5
                            parent_id = level3objectid
                            selected = 0
                            branch_level = 4
                            child_id = id
                            scale_x = .35 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = -75
                            translate_y = 0
                            translate_z = 0
                            rotate_x = 90
                            rotate_y = 0
                            rotate_z = 0
                            ratio = .1
                            line_width= 1
                            geometry = 19
                            topo = 4
                            if row2['C4_Restrictions on gatherings'] == 0:
                                color_r = 230
                                color_g = 234
                                color_b = 234
                                color_a = 255
                            elif row2['C4_Restrictions on gatherings'] == 1:
                                color_r = 174
                                color_g = 200
                                color_b = 238
                                color_a = 255
                            elif row2['C4_Restrictions on gatherings'] == 2:
                                color_r = 108
                                color_g = 127
                                color_b = 230
                                color_a = 255
                            elif row2['C4_Restrictions on gatherings'] == 3:
                                color_r = 65
                                color_g = 48
                                color_b = 214
                                color_a = 255
                            elif row2['C4_Restrictions on gatherings'] == 4:
                                color_r = 68
                                color_g = 0
                                color_b = 199
                                color_a = 255
                            elif row2['C4_Restrictions on gatherings'] == 5:
                                color_r = 59
                                color_g = 196
                                color_b = 255
                                color_a = 0
                            
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)
                            tagtext = "C4_Restrictions on gatherings" + str(row2['C4_Notes'])
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1 
                        elif k == 4:
                            id += 1
                            data = id
                            type = 5
                            parent_id = level3objectid
                            selected = 0
                            branch_level = 4
                            child_id = id
                            scale_x = .35 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = -90
                            translate_y = 0
                            translate_z = 0
                            rotate_x = 90
                            rotate_y = 0
                            rotate_z = 0
                            ratio = .1
                            line_width= 1
                            geometry = 19
                            topo = 4
                            if row2['C5_Close public transport'] == 0:
                                color_r = 230
                                color_g = 234
                                color_b = 234
                                color_a = 255
                            elif row2['C5_Close public transport'] == 1:
                                color_r = 108
                                color_g = 127
                                color_b = 230
                                color_a = 255
                            elif row2['C5_Close public transport'] == 2:
                                color_r = 68
                                color_g = 0
                                color_b = 199
                                color_a = 255
                            elif row2['C5_Close public transport'] == 5:
                                color_r = 59
                                color_g = 196
                                color_b = 255
                                color_a = 0
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)
                            tagtext = "C5_Close public transport" + str(row2['C5_Notes'])
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1
                        elif k == 5:
                            id += 1
                            data = id
                            type = 5
                            parent_id = level3objectid
                            selected = 0
                            branch_level = 4
                            child_id = id
                            scale_x = .35 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = -105
                            translate_y = 0
                            translate_z = 0
                            rotate_x = 90
                            rotate_y = 0
                            rotate_z = 0
                            ratio = .1
                            line_width= 1
                            geometry = 19
                            topo = 4
                            if row2['C6_Stay at home requirements'] == 0:
                                color_r = 230
                                color_g = 234
                                color_b = 234
                                color_a = 255
                            elif row2['C6_Stay at home requirements'] == 1:
                                color_r = 108
                                color_g = 127
                                color_b = 230
                                color_a = 255
                            elif row2['C6_Stay at home requirements'] == 2:
                                color_r = 108
                                color_g = 127
                                color_b = 230
                                color_a = 255
                            elif row2['C6_Stay at home requirements'] == 3:
                                color_r = 68
                                color_g = 0
                                color_b = 199
                                color_a = 255
                            elif row2['C6_Stay at home requirements'] == 5:
                                color_r = 59
                                color_g = 196
                                color_b = 255
                                color_a = 0
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)
                            tagtext = "C6 Stay at home requirements: " + str(row2['C6_Notes'])
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1
                        elif k == 6:
                            id += 1
                            data = id
                            type = 5
                            parent_id = level3objectid
                            selected = 0
                            branch_level = 4
                            child_id = id
                            scale_x = .35 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = -120
                            translate_y = 0
                            translate_z = 0
                            rotate_x = 90
                            rotate_y = 0
                            rotate_z = 0
                            ratio = .1
                            line_width= 1
                            geometry = 19
                            topo = 4
                            if row2['C7_Restrictions on internal movement'] == 0:
                                color_r = 230
                                color_g = 234
                                color_b = 234
                                color_a = 255
                            elif row2['C7_Restrictions on internal movement'] == 1:
                                color_r = 108
                                color_g = 127
                                color_b = 230
                                color_a = 255
                            elif row2['C7_Restrictions on internal movement'] == 2:
                                color_r = 68
                                color_g = 0
                                color_b = 199
                                color_a = 255
                            elif row2['C7_Restrictions on internal movement'] == 5:
                                color_r = 59
                                color_g = 196
                                color_b = 255
                                color_a = 0
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)
                            tagtext = "C7_Restrictions on internal movement" + str(row2['C7_Notes'])
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1
                        elif k == 7:
                            id += 1
                            data = id
                            type = 5
                            parent_id = level3objectid
                            selected = 0
                            branch_level = 4
                            child_id = id
                            scale_x = .35 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = -135
                            translate_y = 0
                            translate_z = 0
                            rotate_x = 90
                            rotate_y = 0
                            rotate_z = 0
                            ratio = .1
                            line_width= 1
                            geometry = 19
                            topo = 4
                            if row2['C8_International travel controls'] == 0:
                                color_r = 230
                                color_g = 234
                                color_b = 234
                                color_a = 255
                            elif row2['C8_International travel controls'] == 1:
                                color_r = 174
                                color_g = 200
                                color_b = 238
                                color_a = 255
                            elif row2['C8_International travel controls'] == 2:
                                color_r = 108
                                color_g = 127
                                color_b = 230
                                color_a = 255
                            elif row2['C8_International travel controls'] == 3:
                                color_r = 65
                                color_g = 48
                                color_b = 214
                                color_a = 255
                            elif row2['C8_International travel controls'] == 4:
                                color_r = 68
                                color_g = 0
                                color_b = 199
                                color_a = 255
                            elif row2['C8_International travel controls'] == 5:
                                color_r = 59
                                color_g = 196
                                color_b = 255
                                color_a = 0
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)
                            tagtext = "C8 International travel controls" + str(row2['C8_Notes'])
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1
                        # red geometry alerting to change in policy
                        elif k == 8:
                            id += 1
                            data = id
                            type = 5
                            parent_id = level3objectid
                            selected = 0
                            branch_level = 4
                            child_id = id
                            scale_x = 1.75 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = 170
                            translate_y = 0
                            translate_z = 0
                            rotate_x = 0
                            rotate_y = 0
                            rotate_z = 0
                            ratio = .1
                            line_width= 1
                            geometry = 15
                            topo = 2
                            color_r = 194
                            color_g = 0
                            color_b = 61
                            if row2[['C1_chg', 'C2_chg', 'C3_chg', 'C4_chg', 'C5_chg', 'C6_chg', 'C7_chg', 'C8_chg']].any() > 0:
                                color_a = 255
                            else:
                                color_a = 0
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)
                            tagtext = "Policy change today"
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1 
                        elif k == 9:
                            id += 1
                            data = id
                            type = 5
                            parent_id = level3objectid
                            selected = 0
                            branch_level = 4
                            child_id = id
                            scale_x = 1.2 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = 25
                            translate_y = 0
                            translate_z = 0
                            rotate_x = 90
                            rotate_y = 90
                            rotate_z = 0
                            ratio = .02
                            line_width= 1
                            geometry = 7
                            topo = 3
                            color_r = 59
                            color_g = 196
                            color_b = 255
                            color_a = 50
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)
                            tagtext = ""
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1
                            level4objectid = id
                            numlevel5toroids = 3

                            for m in range(numlevel5toroids): 
                                if m == 0:
                                    id += 1
                                    data = id
                                    type = 5
                                    parent_id = level4objectid
                                    selected = 0
                                    branch_level = 5
                                    child_id = id
                                    scale_x = row2['move_home_scale'] 
                                    scale_y = scale_x
                                    scale_z = scale_x
                                    translate_x = -120
                                    translate_y = 0
                                    translate_z = 0
                                    rotate_x = 0
                                    rotate_y = 0
                                    rotate_z = 0
                                    ratio = .08
                                    line_width= 1
                                    geometry = 3
                                    topo = 2
                                    color_r = 97
                                    color_g = 29
                                    color_b = 6
                                    color_a = 255
                                    record_id = id 
                                    texture_id= 0

                                    # Creat output string and write to ANTz Node File
                                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                                    fout.write(outputstring)
                                    # Output to Tag file
                                    tagtext = "move at home %: " + str(row2["move_home"])
                                    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                                    ftag.write(tagstring)
                                    taginc += 1
                                elif m == 1:
                                    id += 1
                                    data = id
                                    type = 5
                                    parent_id = level4objectid
                                    selected = 0
                                    branch_level = 5
                                    child_id = id
                                    scale_x = row2['move_partw_scale'] 
                                    scale_y = scale_x
                                    scale_z = scale_x
                                    translate_x = 180
                                    translate_y = 0
                                    translate_z = 0
                                    rotate_x = 0
                                    rotate_y = 0
                                    rotate_z = 0
                                    ratio = .08
                                    line_width= 1
                                    geometry = 3
                                    topo = 2
                                    color_r = 139
                                    color_g = 116
                                    color_b = 255
                                    color_a = 255
                                    record_id = id 
                                    texture_id= 0

                                    # Creat output string and write to ANTz Node File
                                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                                    fout.write(outputstring)
                                    # Output to Tag file
                                    tagtext = "move away part_time: " + str(row2["move_partw"])
                                    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                                    ftag.write(tagstring)
                                    taginc += 1
                                elif m == 2:
                                    id += 1
                                    data = id
                                    type = 5
                                    parent_id = level4objectid
                                    selected = 0
                                    branch_level = 5
                                    child_id = id
                                    scale_x = row2['move_fullw_scale'] 
                                    scale_y = scale_x
                                    scale_z = scale_x
                                    translate_x = 120
                                    translate_y = 0
                                    translate_z = 0
                                    rotate_x = 0
                                    rotate_y = 0
                                    rotate_z = 0
                                    ratio = .08
                                    line_width= 1
                                    geometry = 3
                                    topo = 2
                                    color_r = 255
                                    color_g = 0
                                    color_b = 255
                                    color_a = 255
                                    record_id = id 
                                    texture_id= 0

                                    # Creat output string and write to ANTz Node File
                                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                                    fout.write(outputstring)
                                    # Output to Tag file
                                    tagtext = "move away full time: " + str(row2["move_fullw"])
                                    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                                    ftag.write(tagstring)
                                    taginc += 1
                # search data and public info campaigns    
                elif j == 4:
                    id += 1
                    data = id
                    type = 5
                    parent_id = level2objectid
                    selected = 0
                    branch_level = 3
                    child_id = id
                    translate_x = 0
                    translate_y = 0
                    translate_z = 0
                    scale_x = 1.5
                    scale_y = scale_x
                    scale_z = scale_x
                    line_width= 1
                    geometry = 19
                    topo = 6
                    color_r = 59
                    color_g = 196
                    color_b = 255
                    color_a = 100
                    rotate_x = 0
                    rotate_y = 0
                    rotate_z = 0
                    ratio = .05
                    record_id = id 
                    texture_id= 0
                    hide = 1

                    # Creat output string and write to ANTz Node File
                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                    fout.write(outputstring)
                    tagtext = ""
                    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                    ftag.write(tagstring)
                    taginc += 1
                    level3objectid = id
                    numlevel4toroids = 3
                    for k in range(numlevel4toroids): 
                        if k == 0:
                            id += 1
                            data = id
                            type = 5
                            parent_id = level3objectid
                            selected = 0
                            branch_level = 4
                            child_id = id
                            scale_x = .35 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = -30
                            translate_y = 0
                            translate_z = 0
                            rotate_x = 90
                            rotate_y = 0
                            rotate_z = 0
                            ratio = .1
                            line_width= 1
                            geometry = 19
                            topo = 4
                            if row2['H1_Public information campaigns'] == 0:
                                color_r = 230
                                color_g = 234
                                color_b = 234
                                color_a = 255
                            elif row2['H1_Public information campaigns'] == 1:
                                color_r = 108
                                color_g = 127
                                color_b = 230
                                color_a = 255
                            elif row2['H1_Public information campaigns'] == 2:
                                color_r = 68
                                color_g = 0
                                color_b = 199
                                color_a = 255
                            elif row2['H1_Public information campaigns'] == 5:
                                color_r = 59
                                color_g = 196
                                color_b = 255
                                color_a = 0
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)
                            tagtext = "H1 Public information campaigns: " + str(row2['H1_Notes'])
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1 
                        
                        # red geometry alerting to change in policy
                        elif k == 1:
                            id += 1
                            data = id
                            type = 5
                            parent_id = level3objectid
                            selected = 0
                            branch_level = 4
                            child_id = id
                            scale_x = 1.75 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = 170
                            translate_y = 0
                            translate_z = 0
                            rotate_x = 0
                            rotate_y = 0
                            rotate_z = 0
                            ratio = .1
                            line_width= 1
                            geometry = 15
                            topo = 2
                            color_r = 194
                            color_g = 0
                            color_b = 61
                            if row2['H1_chg'] > 0:
                                color_a = 255
                            else:
                                color_a = 0
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)
                            tagtext = "Policy change today"
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1 
                        elif k == 2:
                            id += 1
                            data = id
                            type = 5
                            parent_id = level3objectid
                            selected = 0
                            branch_level = 4
                            child_id = id
                            scale_x = 1.2 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = 25
                            translate_y = 0
                            translate_z = 0
                            rotate_x = 90
                            rotate_y = 90
                            rotate_z = 0
                            ratio = .02
                            line_width= 1
                            geometry = 7
                            topo = 3
                            color_r = 59
                            color_g = 196
                            color_b = 255
                            color_a = 50
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)
                            tagtext = ""
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1
                            level4objectid = id
                            numlevel5toroids = 1
                            for m in range(numlevel5toroids): 
                                if m == 0:
                                    id += 1
                                    data = id
                                    type = 5
                                    parent_id = level4objectid
                                    selected = 0
                                    branch_level = 5
                                    child_id = id
                                    scale_x = row2['search_scale'] 
                                    scale_y = scale_x
                                    scale_z = scale_x
                                    translate_x = 180
                                    translate_y = 0
                                    translate_z = 0
                                    rotate_x = 0
                                    rotate_y = 0
                                    rotate_z = 0
                                    ratio = .08
                                    line_width= 1
                                    geometry = 3
                                    topo = 2
                                    color_r = 97
                                    color_g = 29
                                    color_b = 6
                                    color_a = 255
                                    record_id = id 
                                    texture_id= 0

                                    # Creat output string and write to ANTz Node File
                                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                                    fout.write(outputstring)
                                    # Output to Tag file
                                    tagtext = "search terms related to covid-19 %: " + str(row2["search"])
                                    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                                    ftag.write(tagstring)
                                    taginc += 1


if row['weekday'] == 1:
        color_r = 255
        color_g = 250
        color_b = 240
    if row['weekday'] == 2:
        color_r = 205
        color_g = 92
        color_b = 92
    if row['weekday'] == 3:
        color_r = 50
        color_g = 205
        color_b = 50
    if row['weekday'] == 4:
        color_r = 255
        color_g = 255
        color_b = 224
    if row['weekday'] == 5:
        color_r = 240
        color_g = 248
        color_b = 255
    if row['weekday'] == 6:
        color_r = 102
        color_g = 51
        color_b = 153
    if row['weekday'] == 0:
        color_r = 128
        color_g = 0
        color_b = 0